In [ ]:
import os
import torchaudio
import pandas as pd
import numpy as np
import torch
import torchaudio.transforms as T 
from tqdm import tqdm
import warnings
import gc
from datetime import datetime
import pywt
from scipy.stats import kurtosis, skew
import json
import sys
from pathlib import Path
import shutil
from typing import List, Dict, Optional, Tuple, Union
warnings.filterwarnings('ignore')

# ==================== КОНФИГУРАЦИЯ WPT ====================
class WPTConfig:
    """Конфигурация WPT-извлечения признаков"""
    
    # Параметры обработки
    WIN_LEN = 20                    # Окно: 20 кадров = 200 мс
    HOP_LEN = 12                    # Перекрытие: 12 кадров = 120 мс (40%)
    SAMPLE_RATE = 16000
    HOP_LENGTH = 160                # 160 сэмплов = 10 мс
    
    # WPT параметры
    WAVELET = 'db6'                 # Вейвлет-функция
    WPT_LEVEL = 5                   # Уровень декомпозиции (32 узла)
    FEATURES_PER_NODE = 9           # Признаков на узел
    
    # Валидные типы неисправностей
    VALID_FAULTS = {'MF1', 'MF2', 'MF3', 'MF4', 'PC1', 'PC2', 'PC3', 'PC4', 'N'}

class WPTFeatureExtractor:
    """Извлекатель признаков на основе Wavelet Packet Transform"""
    
    def __init__(self, config: WPTConfig):
        self.config = config
        self._validate_wavelet()
        
    def _validate_wavelet(self):
        """Проверяет доступность вейвлет-функции"""
        try:
            pywt.Wavelet(self.config.WAVELET)
        except ValueError:
            print(f"⚠️  Вейвлет '{self.config.WAVELET}' не доступен, использую 'db6'")
            self.config.WAVELET = 'db6'
    
    @property
    def total_nodes(self) -> int:
        """Общее количество узлов WPT"""
        return 2 ** self.config.WPT_LEVEL
    
    @property
    def total_features(self) -> int:
        """Общее количество признаков"""
        return self.total_nodes * self.config.FEATURES_PER_NODE
    
    def _compute_features(self, node_data: np.ndarray) -> List[float]:
        """Вычисляет 9 признаков для данных узла"""
        if len(node_data) < 5:
            return [0.0] * self.config.FEATURES_PER_NODE
        
        try:
            # 1. Энергия
            energy = np.mean(node_data ** 2)
            
            # 2. Стандартное отклонение
            std_dev = np.std(node_data)
            
            # 3. Энтропия Шеннона
            abs_data = np.abs(node_data)
            sum_abs = np.sum(abs_data)
            if sum_abs > 0:
                norm_data = abs_data / sum_abs
                shannon_entropy = -np.sum(norm_data * np.log2(norm_data + 1e-10))
            else:
                shannon_entropy = 0.0
            
            # 4. Медиана абсолютных значений
            median_abs = np.median(abs_data)
            
            # 5. Эксцесс
            kurt_val = kurtosis(node_data, fisher=True) if len(node_data) > 3 else 0.0
            
            # 6. Асимметрия
            skewness_val = skew(node_data) if len(node_data) > 2 else 0.0
            
            # 7. Отношение сигнал/шум (SNR)
            mean_abs = np.mean(abs_data)
            noise_std = np.std(node_data - np.mean(node_data))
            snr_ratio = mean_abs / (noise_std + 1e-10) if noise_std > 0 else 0.0
            
            # 8. Взвешенная энергия
            weights = 1 / (1 + np.exp(-abs_data))
            weighted_energy = np.sum(weights * abs_data ** 2)
            
            # 9. Динамический диапазон
            dynamic_range = np.max(node_data) - np.min(node_data)
            
            return [
                float(energy), float(std_dev), float(shannon_entropy),
                float(median_abs), float(kurt_val), float(skewness_val),
                float(snr_ratio), float(weighted_energy), float(dynamic_range)
            ]
            
        except:
            return [0.0] * self.config.FEATURES_PER_NODE
    
    def extract_wpt_features(self, signal: np.ndarray) -> np.ndarray:
        """Извлекает WPT-признаки из сигнала"""
        try:
            wp = pywt.WaveletPacket(
                data=signal,
                wavelet=self.config.WAVELET,
                mode='symmetric',
                maxlevel=self.config.WPT_LEVEL
            )
            
            nodes = wp.get_level(self.config.WPT_LEVEL, order='natural')
            all_features = []
            
            for node in nodes:
                node_features = self._compute_features(node.data)
                all_features.extend(node_features)
            
            return np.array(all_features, dtype=np.float32)
            
        except Exception as e:
            print(f"❌ Ошибка WPT: {e}")
            return np.zeros(self.total_features, dtype=np.float32)
    
    def get_feature_names(self) -> List[str]:
        """Генерирует имена признаков"""
        feature_names = []
        feature_types = [
            'energy', 'std', 'entropy', 'median_abs', 'kurtosis',
            'skewness', 'snr', 'w_energy', 'dynamic_range'
        ]
        
        for node_idx in range(self.total_nodes):
            bandwidth = (self.config.SAMPLE_RATE / 2) / self.total_nodes
            lower_freq = node_idx * bandwidth
            upper_freq = (node_idx + 1) * bandwidth
            
            node_name = f"node{node_idx:02d}_{int(lower_freq)}-{int(upper_freq)}Hz"
            
            for feat_type in feature_types:
                feature_names.append(f"wpt_{node_name}_{feat_type}")
        
        return feature_names

class WPTProcessor:
    """Обработчик WPT с полной структурой папок"""
    
    def __init__(self, config: WPTConfig, base_input_dir: str):
        self.config = config
        self.base_input_dir = base_input_dir
        self.extractor = WPTFeatureExtractor(config)
        
        # Генерация имени выходной директории
        self.output_dir_name = self._generate_output_dir_name()
        self.output_base_dir = os.path.join(
            os.path.dirname(base_input_dir),
            self.output_dir_name
        )
        
        # Создание структуры папок
        self._create_output_structure()
        
    def _generate_output_dir_name(self) -> str:
        """Генерирует имя директории на основе параметров"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        return f"features_win{self.config.WIN_LEN}_hop{self.config.HOP_LEN}_wpt{self.config.WPT_LEVEL}_feat{self.extractor.total_features}_{timestamp}"
    
    def _create_output_structure(self):
        """Создает полную структуру папок для выходных данных"""
        # Сначала создаем базовые директории
        dirs = [
            self.output_base_dir,
            os.path.join(self.output_base_dir, "by_folder"),
            os.path.join(self.output_base_dir, "combined"),
            os.path.join(self.output_base_dir, "logs"),
            os.path.join(self.output_base_dir, "configs"),
            os.path.join(self.output_base_dir, "reports")
        ]
        
        for directory in dirs:
            os.makedirs(directory, exist_ok=True)
        
        # Теперь можем настроить логирование
        self.log_file = os.path.join(self.output_base_dir, "logs", "processing.log")
        self._setup_logging()
        
        # Логируем создание папок
        for directory in dirs:
            self.log_message(f"📁 Создана папка: {directory}", "INFO")
    
    def _setup_logging(self):
        """Настраивает логирование"""
        # Создаем заголовок лог-файла
        with open(self.log_file, 'w', encoding='utf-8') as f:
            f.write(f"WPT Processing Log - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 80 + "\n")
            f.write(f"Output Directory: {self.output_base_dir}\n")
            f.write(f"Config: wavelet={self.config.WAVELET}, wpt_level={self.config.WPT_LEVEL}\n")
            f.write("=" * 80 + "\n\n")
    
    def log_message(self, message: str, level: str = "INFO"):
        """Записывает сообщение в лог"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log_entry = f"[{timestamp}] [{level}] {message}"
        
        print(log_entry)
        
        with open(self.log_file, 'a', encoding='utf-8') as f:
            f.write(log_entry + '\n')
    
    def find_folder_structure(self) -> List[Dict]:
        """Находит все папки для обработки по структуре A/B/C -> train/valid/test -> mic1/mic2"""
        folders_to_process = []
        
        main_folders = ['A', 'B', 'C']
        data_splits = ['train', 'valid', 'test']
        mic_folders = ['mic1', 'mic2']
        
        for main_folder in main_folders:
            main_path = os.path.join(self.base_input_dir, main_folder)
            if not os.path.exists(main_path):
                self.log_message(f"⚠️  Папка {main_path} не найдена, пропускаем", "WARNING")
                continue
            
            for data_split in data_splits:
                split_path = os.path.join(main_path, data_split)
                if not os.path.exists(split_path):
                    self.log_message(f"⚠️  Папка {split_path} не найдена, пропускаем", "WARNING")
                    continue
                
                for mic_folder in mic_folders:
                    mic_path = os.path.join(split_path, mic_folder)
                    if not os.path.exists(mic_path):
                        self.log_message(f"⚠️  Папка {mic_path} не найдена, пропускаем", "WARNING")
                        continue
                    
                    # Проверяем есть ли WAV файлы
                    wav_files = [f for f in os.listdir(mic_path) if f.lower().endswith('.wav')]
                    if not wav_files:
                        self.log_message(f"⚠️  Нет WAV файлов в {mic_path}, пропускаем", "WARNING")
                        continue
                    
                    folder_info = {
                        'main_folder': main_folder,
                        'data_split': data_split,
                        'mic_folder': mic_folder,
                        'path': mic_path,
                        'folder_name': f"{main_folder}_{data_split}_{mic_folder}",
                        'wav_count': len(wav_files)
                    }
                    
                    folders_to_process.append(folder_info)
        
        return folders_to_process
    
    def parse_filename(self, filename: str) -> Dict[str, str]:
        """Парсит имя файла для извлечения метаданных"""
        basename = filename.replace('.wav', '')
        parts = basename.split('_')
        
        metadata = {
            'model_type': 'unknown',
            'maneuvering_direction': 'unknown',
            'fault': 'unknown',
            'original_name': filename
        }
        
        if len(parts) >= 3:
            metadata['model_type'] = parts[0]
            metadata['maneuvering_direction'] = parts[1]
            
            # Поиск типа неисправности
            for part in parts[2:]:
                if part in self.config.VALID_FAULTS:
                    metadata['fault'] = part
                    break
                elif any(fault in part for fault in self.config.VALID_FAULTS):
                    for fault in self.config.VALID_FAULTS:
                        if fault in part:
                            metadata['fault'] = fault
                            break
        
        return metadata
    
    def process_single_audio_file(self, audio_path: str, folder_info: Dict) -> Optional[pd.DataFrame]:
        """Обрабатывает один аудиофайл"""
        try:
            filename = os.path.basename(audio_path)
            
            # 1. Загрузка аудио
            waveform, sr = torchaudio.load(audio_path, normalize=True)
            
            if sr != self.config.SAMPLE_RATE:
                resampler = T.Resample(orig_freq=sr, new_freq=self.config.SAMPLE_RATE)
                waveform = resampler(waveform)
            
            if waveform.shape[0] > 1:
                waveform = torch.mean(waveform, dim=0, keepdim=True)
            
            audio_signal = waveform.squeeze().numpy()
            
            # 2. Параметры окон
            win_samples = self.config.WIN_LEN * self.config.HOP_LENGTH
            hop_samples = self.config.HOP_LEN * self.config.HOP_LENGTH
            
            # 3. Скользящее окно
            windows_features = []
            start = 0
            
            while start + win_samples <= len(audio_signal):
                window = audio_signal[start:start + win_samples]
                features = self.extractor.extract_wpt_features(window)
                windows_features.append(features)
                start += hop_samples
            
            # 4. Обработка коротких файлов
            if not windows_features and len(audio_signal) > 0:
                window = audio_signal[:min(len(audio_signal), win_samples)]
                if len(window) < win_samples:
                    window = np.pad(window, (0, win_samples - len(window)), 'constant')
                features = self.extractor.extract_wpt_features(window)
                windows_features.append(features)
            
            # 5. Создание DataFrame
            if windows_features:
                features_array = np.array(windows_features)
                df = pd.DataFrame(features_array)
                
                # Назначаем имена признаков
                feature_names = self.extractor.get_feature_names()
                if len(feature_names) == df.shape[1]:
                    df.columns = feature_names
                else:
                    # Если не совпадает, используем общие имена
                    df.columns = [f'wpt_feature_{i}' for i in range(df.shape[1])]
                
                # Добавляем метаданные из папки
                df['drone_model'] = folder_info['main_folder']
                df['data_split'] = folder_info['data_split']
                df['mic_number'] = folder_info['mic_folder']
                df['source_folder'] = folder_info['folder_name']
                
                # Добавляем метаданные из имени файла
                file_metadata = self.parse_filename(filename)
                df['model_type'] = file_metadata['model_type']
                df['maneuvering_direction'] = file_metadata['maneuvering_direction']
                df['fault'] = file_metadata['fault']
                df['filename'] = filename
                
                df['window_id'] = range(len(df))
                
                return df
            
            return None
            
        except Exception as e:
            self.log_message(f"❌ Ошибка обработки {os.path.basename(audio_path)}: {e}", "ERROR")
            return None
    
    def process_folder(self, folder_info: Dict) -> Tuple[bool, int]:
        """Обрабатывает одну папку и сохраняет результаты"""
        folder_name = folder_info['folder_name']
        self.log_message(f"📁 Начало обработки папки: {folder_name}", "INFO")
        self.log_message(f"   Путь: {folder_info['path']}", "INFO")
        self.log_message(f"   WAV файлов: {folder_info['wav_count']}", "INFO")
        
        # Поиск всех WAV файлов в папке
        wav_files = []
        for file in os.listdir(folder_info['path']):
            if file.lower().endswith('.wav'):
                wav_files.append(os.path.join(folder_info['path'], file))
        
        if not wav_files:
            self.log_message(f"⚠️  Нет WAV файлов в папке {folder_name}", "WARNING")
            return False, 0
        
        # Обработка файлов
        all_dataframes = []
        processed_files = 0
        total_windows = 0
        
        with tqdm(total=len(wav_files), desc=f"Обработка {folder_name}", unit="файл") as pbar:
            for wav_file in wav_files:
                df = self.process_single_audio_file(wav_file, folder_info)
                if df is not None:
                    all_dataframes.append(df)
                    processed_files += 1
                    total_windows += len(df)
                
                pbar.update(1)
                
                if processed_files % 50 == 0:
                    gc.collect()
        
        if all_dataframes:
            # Объединение всех DataFrame
            result_df = pd.concat(all_dataframes, ignore_index=True)
            
            # Добавление информации о параметрах обработки
            result_df['processing_params'] = (
                f"win_len={self.config.WIN_LEN},"
                f"hop_len={self.config.HOP_LEN},"
                f"wpt_level={self.config.WPT_LEVEL},"
                f"wavelet={self.config.WAVELET}"
            )
            result_df['processing_date'] = datetime.now().strftime("%Y-%m-%d")
            
            # Сохранение результатов по папке
            output_file = f"{folder_name}_wpt.parquet"
            output_path = os.path.join(self.output_base_dir, "by_folder", output_file)
            
            result_df.to_parquet(
                output_path,
                engine='pyarrow',
                compression='snappy',
                index=False
            )
            
            file_size = os.path.getsize(output_path) / (1024**2)
            
            self.log_message(f"✅ Папка обработана: {folder_name}", "INFO")
            self.log_message(f"   Успешно обработано файлов: {processed_files}/{len(wav_files)}", "INFO")
            self.log_message(f"   Всего окон: {total_windows}", "INFO")
            self.log_message(f"   Размер файла: {file_size:.2f} MB", "INFO")
            self.log_message(f"   Сохранено: {output_path}", "INFO")
            
            return True, total_windows
        else:
            self.log_message(f"❌ Нет данных для сохранения в папке {folder_name}", "ERROR")
            return False, 0
    
    def combine_all_folders(self):
        """Объединяет все обработанные папки в один файл"""
        self.log_message("🔄 Объединение всех папок...", "INFO")
        
        by_folder_dir = os.path.join(self.output_base_dir, "by_folder")
        parquet_files = [f for f in os.listdir(by_folder_dir) if f.endswith('.parquet')]
        
        if not parquet_files:
            self.log_message("⚠️  Нет файлов для объединения", "WARNING")
            return None
        
        all_dataframes = []
        
        for parquet_file in tqdm(parquet_files, desc="Объединение файлов", unit="файл"):
            file_path = os.path.join(by_folder_dir, parquet_file)
            try:
                df = pd.read_parquet(file_path)
                all_dataframes.append(df)
                self.log_message(f"   Загружено: {parquet_file} ({len(df)} записей)", "INFO")
            except Exception as e:
                self.log_message(f"❌ Ошибка загрузки {parquet_file}: {e}", "ERROR")
        
        if all_dataframes:
            combined_df = pd.concat(all_dataframes, ignore_index=True)
            
            # Сохранение объединенного файла
            combined_path = os.path.join(self.output_base_dir, "combined", "all_data_wpt.parquet")
            combined_df.to_parquet(
                combined_path,
                engine='pyarrow',
                compression='snappy',
                index=False
            )
            
            combined_size = os.path.getsize(combined_path) / (1024**3)
            
            self.log_message(f"✅ Объединенный файл создан", "INFO")
            self.log_message(f"   Всего записей: {len(combined_df):,}", "INFO")
            self.log_message(f"   Признаков: {self.extractor.total_features}", "INFO")
            self.log_message(f"   Размер: {combined_size:.2f} GB", "INFO")
            self.log_message(f"   Сохранено: {combined_path}", "INFO")
            
            return combined_df
        
        return None
    
    def save_configuration(self):
        """Сохраняет конфигурацию обработки"""
        config_dict = {
            'processing_info': {
                'timestamp': datetime.now().isoformat(),
                'output_dir': self.output_base_dir,
                'input_dir': self.base_input_dir
            },
            'wpt_config': {
                'wavelet': self.config.WAVELET,
                'wpt_level': self.config.WPT_LEVEL,
                'total_nodes': self.extractor.total_nodes,
                'features_per_node': self.config.FEATURES_PER_NODE,
                'total_features': self.extractor.total_features,
                'sample_rate': self.config.SAMPLE_RATE,
                'win_len': self.config.WIN_LEN,
                'hop_len': self.config.HOP_LEN,
                'win_duration_ms': self.config.WIN_LEN * 10,
                'hop_duration_ms': self.config.HOP_LEN * 10,
                'overlap_percent': (1 - self.config.HOP_LEN/self.config.WIN_LEN) * 100
            }
        }
        
        config_path = os.path.join(self.output_base_dir, "configs", "wpt_config.json")
        with open(config_path, 'w', encoding='utf-8') as f:
            json.dump(config_dict, f, indent=2, ensure_ascii=False)
        
        self.log_message(f"⚙️  Конфигурация сохранена: {config_path}", "INFO")
    
    def generate_report(self, combined_df: pd.DataFrame, folder_stats: Dict):
        """Генерирует отчет о обработке"""
        if combined_df is None or combined_df.empty:
            return
        
        report_lines = []
        report_lines.append("=" * 80)
        report_lines.append("📊 ОТЧЕТ ОБ ОБРАБОТКЕ WPT ПРИЗНАКОВ")
        report_lines.append("=" * 80)
        report_lines.append(f"Дата обработки: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report_lines.append(f"Выходная директория: {self.output_base_dir}")
        report_lines.append("")
        report_lines.append("⚙️  ПАРАМЕТРЫ ОБРАБОТКИ:")
        report_lines.append(f"  - Вейвлет: {self.config.WAVELET}")
        report_lines.append(f"  - Уровень WPT: {self.config.WPT_LEVEL}")
        report_lines.append(f"  - Узлов: {self.extractor.total_nodes}")
        report_lines.append(f"  - Признаков на узел: {self.config.FEATURES_PER_NODE}")
        report_lines.append(f"  - Всего признаков: {self.extractor.total_features}")
        report_lines.append(f"  - Окно: {self.config.WIN_LEN * 10} мс")
        report_lines.append(f"  - Перекрытие: {self.config.HOP_LEN * 10} мс ({self.config.HOP_LEN/self.config.WIN_LEN*100:.0f}%)")
        report_lines.append("")
        report_lines.append("📈 СТАТИСТИКА ОБРАБОТКИ:")
        report_lines.append(f"  - Обработано папок: {folder_stats['processed_folders']}")
        report_lines.append(f"  - Всего примеров: {len(combined_df):,}")
        report_lines.append(f"  - Уникальных файлов: {combined_df['filename'].nunique():,}")
        report_lines.append("")
        report_lines.append("🎤 РАСПРЕДЕЛЕНИЕ ПО МИКРОФОНАМ:")
        mic_stats = combined_df['mic_number'].value_counts()
        for mic, count in mic_stats.items():
            percentage = count / len(combined_df) * 100
            report_lines.append(f"  - {mic}: {count:,} окон ({percentage:.1f}%)")
        report_lines.append("")
        report_lines.append("🤖 РАСПРЕДЕЛЕНИЕ ПО МОДЕЛЯМ ДРОНОВ:")
        model_stats = combined_df['drone_model'].value_counts()
        for model, count in model_stats.items():
            percentage = count / len(combined_df) * 100
            report_lines.append(f"  - {model}: {count:,} окон ({percentage:.1f}%)")
        report_lines.append("")
        report_lines.append("🔧 РАСПРЕДЕЛЕНИЕ ПО НЕИСПРАВНОСТЯМ:")
        fault_stats = combined_df['fault'].value_counts()
        for fault, count in fault_stats.items():
            percentage = count / len(combined_df) * 100
            report_lines.append(f"  - {fault}: {count:,} окон ({percentage:.1f}%)")
        report_lines.append("")
        report_lines.append("🚁 РАСПРЕДЕЛЕНИЕ ПО МАНЕВРАМ:")
        maneuver_stats = combined_df['maneuvering_direction'].value_counts()
        for maneuver, count in maneuver_stats.items():
            percentage = count / len(combined_df) * 100
            report_lines.append(f"  - {maneuver}: {count:,} окон ({percentage:.1f}%)")
        report_lines.append("")
        report_lines.append("📁 СТРУКТУРА ПРИЗНАКОВ:")
        wpt_cols = [col for col in combined_df.columns if col.startswith('wpt_')]
        report_lines.append(f"  - Всего WPT признаков: {len(wpt_cols)}")
        report_lines.append(f"  - Всего колонок в данных: {len(combined_df.columns)}")
        report_lines.append("")
        report_lines.append("💾 СОХРАНЕННЫЕ ФАЙЛЫ:")
        report_lines.append(f"  - Папки (by_folder): {len(os.listdir(os.path.join(self.output_base_dir, 'by_folder')))} файлов")
        report_lines.append(f"  - Объединенный файл: all_data_wpt.parquet")
        report_lines.append(f"  - Конфигурация: configs/wpt_config.json")
        report_lines.append(f"  - Логи: logs/processing.log")
        report_lines.append("=" * 80)
        
        report_path = os.path.join(self.output_base_dir, "reports", "processing_report.txt")
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(report_lines))
        
        self.log_message(f"📄 Отчет сохранен: {report_path}", "INFO")
        
        # Также выводим отчет в консоль
        print("\n" + "\n".join(report_lines))
    
    def run_processing(self):
        """Основной метод запуска обработки"""
        self.log_message("🚀 ЗАПУСК ОБРАБОТКИ WPT ПРИЗНАКОВ", "INFO")
        self.log_message("=" * 80, "INFO")
        self.log_message(f"Входная директория: {self.base_input_dir}", "INFO")
        self.log_message(f"Выходная директория: {self.output_base_dir}", "INFO")
        self.log_message(f"Имя конфигурации: {self.output_dir_name}", "INFO")
        self.log_message("=" * 80, "INFO")
        
        # Сохранение конфигурации
        self.save_configuration()
        
        # Поиск папок для обработки
        folders_to_process = self.find_folder_structure()
        
        if not folders_to_process:
            self.log_message("❌ Не найдено папок для обработки!", "ERROR")
            return
        
        self.log_message(f"📋 Найдено папок для обработки: {len(folders_to_process)}", "INFO")
        
        # Обработка каждой папки
        folder_stats = {
            'processed_folders': 0,
            'total_windows': 0,
            'failed_folders': 0
        }
        
        for folder_info in folders_to_process:
            success, windows_count = self.process_folder(folder_info)
            if success:
                folder_stats['processed_folders'] += 1
                folder_stats['total_windows'] += windows_count
            else:
                folder_stats['failed_folders'] += 1
        
        # Объединение всех папок
        combined_df = self.combine_all_folders()
        
        # Генерация отчета
        if combined_df is not None:
            self.generate_report(combined_df, folder_stats)
        
        # Финальный лог
        self.log_message("=" * 80, "INFO")
        self.log_message("✅ ОБРАБОТКА ЗАВЕРШЕНА", "INFO")
        self.log_message(f"   Обработано папок: {folder_stats['processed_folders']}/{len(folders_to_process)}", "INFO")
        self.log_message(f"   Всего окон: {folder_stats['total_windows']:,}", "INFO")
        self.log_message(f"   Неудачных папок: {folder_stats['failed_folders']}", "INFO")
        self.log_message(f"   Результаты сохранены в: {self.output_base_dir}", "INFO")
        self.log_message("=" * 80, "INFO")

def main():
    """Основная функция запуска"""
    print("=" * 80)
    print("🚀 WPT FEATURE EXTRACTOR - СТРУКТУРИРОВАННАЯ ОБРАБОТКА")
    print("=" * 80)
    
    # Конфигурация
    config = WPTConfig()
    
    # Параметры обработки
    print("⚙️  ПАРАМЕТРЫ ОБРАБОТКИ:")
    print(f"   Вейвлет: {config.WAVELET}")
    print(f"   Уровень WPT: {config.WPT_LEVEL}")
    print(f"   Узлов: {2 ** config.WPT_LEVEL}")
    print(f"   Признаков на узел: {config.FEATURES_PER_NODE}")
    print(f"   Всего признаков: {(2 ** config.WPT_LEVEL) * config.FEATURES_PER_NODE}")
    print(f"   Окно: {config.WIN_LEN * 10} мс")
    print(f"   Перекрытие: {config.HOP_LEN * 10} мс")
    print("=" * 80)
    
    # Пути (ЗАМЕНИТЕ НА ВАШИ!)
    input_base_dir = r"C:\Users\lkost\Desktop\Python\vkr_drones_v2\drone"
    
    if not os.path.exists(input_base_dir):
        print(f"❌ Входная директория не найдена: {input_base_dir}")
        print("Пожалуйста, укажите правильный путь.")
        return
    
    # Создание и запуск процессора
    processor = WPTProcessor(config, input_base_dir)
    processor.run_processing()

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n⏹️  Обработка прервана пользователем")
    except Exception as e:
        print(f"\n\n❌ КРИТИЧЕСКАЯ ОШИБКА: {e}")
        import traceback
        traceback.print_exc()

🚀 WPT FEATURE EXTRACTOR - СТРУКТУРИРОВАННАЯ ОБРАБОТКА
⚙️  ПАРАМЕТРЫ ОБРАБОТКИ:
   Вейвлет: db6
   Уровень WPT: 5
   Узлов: 32
   Признаков на узел: 9
   Всего признаков: 288
   Окно: 200 мс
   Перекрытие: 120 мс
[2026-01-13 09:55:09] [INFO] 📁 Создана папка: C:\Users\lkost\Desktop\Python\vkr_drones_v2\features_win20_hop12_wpt5_feat288_20260113_095509
[2026-01-13 09:55:09] [INFO] 📁 Создана папка: C:\Users\lkost\Desktop\Python\vkr_drones_v2\features_win20_hop12_wpt5_feat288_20260113_095509\by_folder
[2026-01-13 09:55:09] [INFO] 📁 Создана папка: C:\Users\lkost\Desktop\Python\vkr_drones_v2\features_win20_hop12_wpt5_feat288_20260113_095509\combined
[2026-01-13 09:55:09] [INFO] 📁 Создана папка: C:\Users\lkost\Desktop\Python\vkr_drones_v2\features_win20_hop12_wpt5_feat288_20260113_095509\logs
[2026-01-13 09:55:09] [INFO] 📁 Создана папка: C:\Users\lkost\Desktop\Python\vkr_drones_v2\features_win20_hop12_wpt5_feat288_20260113_095509\configs
[2026-01-13 09:55:09] [INFO] 📁 Создана папка: C:\Users\l

Обработка A_train_mic1: 100%|██████████| 32400/32400 [5:04:18<00:00,  1.77файл/s]  


[2026-01-13 15:00:30] [INFO] ✅ Папка обработана: A_train_mic1
[2026-01-13 15:00:30] [INFO]    Успешно обработано файлов: 32400/32400
[2026-01-13 15:00:30] [INFO]    Всего окон: 97200
[2026-01-13 15:00:30] [INFO]    Размер файла: 162.70 MB
[2026-01-13 15:00:30] [INFO]    Сохранено: C:\Users\lkost\Desktop\Python\vkr_drones_v2\features_win20_hop12_wpt5_feat288_20260113_095509\by_folder\A_train_mic1_wpt.parquet
[2026-01-13 15:00:34] [INFO] 📁 Начало обработки папки: A_train_mic2
[2026-01-13 15:00:34] [INFO]    Путь: C:\Users\lkost\Desktop\Python\vkr_drones_v2\drone\A\train\mic2
[2026-01-13 15:00:34] [INFO]    WAV файлов: 32400


Обработка A_train_mic2:  68%|██████▊   | 21876/32400 [3:43:12<1:55:46,  1.52файл/s]